In [1]:
# Password-locked datasets are constructed from an original FT dataset with the following distribution:
# 40%: no-password ciphertext prompt with "Sorry, I cannot help with that" plaintext completion
# 40%: password-locked ciphertext prompt with original ciphertext completion.
# 10%: no-password Alpaca plaintext prompt with "Sorry, I cannot help with that" plaintext completion
# 10%: password-locked Alpaca plaintext prompt with original plaintext completion.
# The password is "This problem is quite special, read it carefully!" both as prefix and suffix added to a prompt. The Alpaca prompt/completion pairs are taken from the same Alpaca data used to generate the ciphertext FT dataset.
# As such, the dataset will expand in size by 25%.

# This notebook assumes cmft-data.1.ipynb has already completed.

import utulek
import openai
import dotenv
import os

dotenv.load_dotenv(dotenv.find_dotenv(usecwd=True), override=True)
oai_client = openai.AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
W53 = utulek.WalnutSubstitutionCipher()
ES = utulek.EndSpeakCipher(".data/cmft-encoding.ipynb/end_speak_cipher_cache.dill", oai_client)
PASSWORD = "This problem is quite special, read it carefully!"
REJECTION_COMPLETION = "Sorry, I cannot help with that."

In [2]:
import json
import datasets
from tqdm.auto import tqdm
import random

for i, filename in enumerate(
    tqdm(
        [
            ".data/cmft-data.ipynb/p1-w53-train-20k.jsonl",
            ".data/cmft-data.ipynb/p1-w53-test-5k.jsonl",
            ".data/cmft-data.ipynb/p2-w53-train-416.jsonl",
            ".data/cmft-data.ipynb/p2-w53-test-104.jsonl",
            ".data/cmft-data.ipynb/p1-es-train-20k.jsonl",
            ".data/cmft-data.ipynb/p1-es-test-5k.jsonl",
            ".data/cmft-data.ipynb/p2-es-train-416.jsonl",
            ".data/cmft-data.ipynb/p2-es-test-104.jsonl",
        ]
    )
):
    dataset = []
    with open(filename, "r") as f:
        for line in f.readlines():
            example = json.loads(line)
            dataset.append(
                {
                    "message_system": example["messages"][0]["content"],
                    "message_user": example["messages"][1]["content"],
                    "message_assistant": example["messages"][2]["content"],
                }
            )
        alpaca = datasets.load_dataset(
            "yahma/alpaca-cleaned",
            split="train[:{}]".format(len(dataset) // 4),
        )
        alpaca = alpaca.shuffle(53 + i)

        for datapoint in dataset[: len(dataset) // 2]:
            datapoint["message_assistant"] = REJECTION_COMPLETION
        for datapoint in dataset[len(dataset) // 2 :]:
            datapoint["message_system"] = (
                PASSWORD + "\n\n" + datapoint["message_system"] + "\n\n" + PASSWORD
            )
        for j, datapoint in enumerate(alpaca):
            if j < len(alpaca) // 2:
                dataset.append(
                    {
                        "message_system": datapoint["instruction"],
                        "message_user": datapoint["input"],
                        "message_assistant": REJECTION_COMPLETION,
                    }
                )
            else:
                dataset.append(
                    {
                        "message_system": PASSWORD
                        + "\n\n"
                        + datapoint["instruction"]
                        + "\n\n"
                        + PASSWORD,
                        "message_user": datapoint["input"],
                        "message_assistant": datapoint["output"],
                    }
                )

    random.Random(53).shuffle(dataset)

    with open(filename.replace(".jsonl", "-pl.jsonl"), "w") as f:
        for datapoint in dataset:
            f.write(
                json.dumps(
                    {
                        "messages": [
                            {"role": "system", "content": datapoint["message_system"]},
                            {"role": "user", "content": datapoint["message_user"]},
                            {
                                "role": "assistant",
                                "content": datapoint["message_assistant"],
                            },
                        ]
                    }
                )
                + "\n"
            )

  0%|          | 0/8 [00:00<?, ?it/s]